In [ ]:
# Imports and definitions

import numpy as np
import os
from enum import Enum
from convert_board import convert_board

class BoardFields(Enum):
    EMPTY = 0
    PLAYER1 = 1
    PLAYER2 = 2

NO_MOVE = -1000


In [ ]:
# Convert data to python arrays

boards = []
evaluations = []

def read_line(line):
    splited_line = line.split(",")
    moves_string = splited_line[0]
    columns_first_empty_square = [0 for _ in range(7)]
    board = np.full((7, 6), BoardFields.EMPTY.value, dtype="int8")
    turn = BoardFields.PLAYER1

    for move_string in moves_string:
        move = int(move_string) - 1
        board[move, columns_first_empty_square[move]] = turn.value
        columns_first_empty_square[move] += 1
        turn = (
            BoardFields.PLAYER1 if turn == BoardFields.PLAYER2 else BoardFields.PLAYER2
        )

    child_boards_evaluations = [int(splited_line[i]) for i in range(1, 8)]

    for i in range(len(child_boards_evaluations)):
        if child_boards_evaluations[i] == NO_MOVE:
            continue
        child_board = np.copy(board)
        child_board[i, columns_first_empty_square[i]] = turn.value
        boards.append(child_board)
        evaluations.append(child_boards_evaluations[i])


for file_name in os.listdir(os.getcwd() + "/data"):
    file_path = os.getcwd() + "/data/" + file_name
    try:
        with open(file_path, "r") as file:
            for line in file:
                read_line(line.strip())
    except FileNotFoundError:
        print("File not found.")
    except IOError:
        print("Error reading the file.")

np.save(os.getcwd() + "/boards.npy", boards)
np.save(os.getcwd() + "/evaluations.npy", evaluations)


In [ ]:
# Convert python arrays into model input

increment = 1

boards = np.load(os.getcwd() + "/boards.npy")
positions = []
player1_lines = []
player2_lines = []

length = len(boards)
count = 0
percentage = 0
for board in boards:
    converted_board = convert_board(board)

    positions.append(converted_board[0])
    player1_lines.append(converted_board[1])
    player2_lines.append(converted_board[2])

    count += 1
    if count > percentage / 100 * length:
        print(percentage, "%")
        percentage += increment

np.save(os.getcwd() + "/positions.npy", positions)
np.save(os.getcwd() + "/player1_lines.npy", player1_lines)
np.save(os.getcwd() + "/player2_lines.npy", player2_lines)


In [ ]:
# Check if data was converted correctly

boards = np.load(os.getcwd() + "/boards.npy")
evaluations = np.load(os.getcwd() + "/evaluations.npy")
player1_lines = np.load(os.getcwd() + "/player1_lines.npy")
player2_lines = np.load(os.getcwd() + "/player2_lines.npy")

print("size:", len(boards))

i = 0
print(boards[i])
print("evaluation:", evaluations[i])
print("player 1 lines:", player1_lines[i])
print("player 2 lines:", player2_lines[i])
